In [5]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq
pd.options.display.max_rows = 5

In [6]:
my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [7]:
end_use = "hs?get=E_COMMODITY,ALL_VAL_MO"

url = "https://api.census.gov/data/timeseries/intltrade/exports/"
url = url + end_use + my_key + "&time==from+2017-01" + "&COMM_LVL=HS10"
# note the change in the commodity level

r = requests.get(url)

In [8]:
df_all_trade = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
# Note that the first entry is the labels

df_all_trade.columns = r.json()[0]

df_all_trade.time = pd.to_datetime(df_all_trade.time, format="%Y-%m")
# This is so I can call this correctly...
df_all_trade.head(10)

# df_all_trade.to_stata('/Users/changjay/Desktop/Park Project/bls_quarterly_county/all_trade.dta')
# this is where I export to stata

df_all_trade["total_trade"] = df_all_trade.ALL_VAL_MO.astype(float)

df_all_trade.E_COMMODITY = df_all_trade.E_COMMODITY.astype(str)

df_all_trade.head(10)

,E_COMMODITY,ALL_VAL_MO,time,COMM_LVL,total_trade
0,0101210000,9552860,2017-01-01,HS10,9552860.0
1,0101290000,10929152,2017-01-01,HS10,10929152.0
...,...,...,...,...,...
8,8707905060,754877,2017-01-01,HS10,754877.0
9,8707905080,1107603,2017-01-01,HS10,1107603.0


In [9]:
df_all_trade.set_index("time", inplace = True)
# use time as index
df_all_trade.head(10)

,E_COMMODITY,ALL_VAL_MO,COMM_LVL,total_trade
time,,,,
2017-01-01,0101210000,9552860,HS10,9552860.0
2017-01-01,0101290000,10929152,HS10,10929152.0
...,...,...,...,...
2017-01-01,8707905060,754877,HS10,754877.0
2017-01-01,8707905080,1107603,HS10,1107603.0


In [10]:
dftrade_17 = df_all_trade.loc["2017"].groupby("E_COMMODITY").agg({"total_trade":"sum"})
# dftrade_17.head()
dftrade_17.shape # Obs: 9,151

(9151, 1)

In [11]:
df_concordance = pd.read_excel("/Users/changjay/Desktop/Pandas-to-STATA Project/alt_hs_naics_mapping/expconcord17.xls", 
                      dtype = {"commodity": str, "naics": str})
# df_concordance.head(10)

df_concordance["hs8"] = df_concordance.commodity.str[0:8]
# truncate down to get the hs8

df_concordance["hs6"] = df_concordance.commodity.str[0:6]
# truncate down to get the hs6
df_concordance.head(10)

#df_concordance["naics3"] = df_concordance["naics"].str[0:3]

#dict_concordance = dict(zip(df_concordance.hs6,df_concordance.naics)) 

# This creates a dictionaty from which we can map the hs6 to the naics codes

,commodity,descriptn,abbreviatn,unit_qy1,unit_qy2,sitc,end_use,naics,usda,hitech,hs8,hs6
0,0101210000,"HORSES, PUREBRED BREEDING, LIVE","HORSES, PUREBRED BREEDING, LIVE",NO,NaN,00150,10140,112920,0,0,01012100,010121
1,0101290000,"HORSES, LIVE, EXCEPT PUREBRED BREEDING","HORSES, LIVE, EXCEPT PUREBRED BREEDING",NO,NaN,00150,10140,112920,0,0,01012900,010129
...,...,...,...,...,...,...,...,...,...,...,...,...
8,0102290000,"CATTLE, LIVE, OTHER THAN PUREBRED BREEDING ANI...","CATTLE, LIVE, OTHER THAN PUREBRED BREEDING ANI...",NO,NaN,00119,300,11211X,0,0,01022900,010229
9,0102310000,"BUFFALO, FOR PUREBRED BREEDING, LIVE","BUFFALO, FOR PUREBRED BREEDING, LIVE",NO,NaN,00111,10140,11211X,0,0,01023100,010231


In [16]:
trade_conc = dftrade_17.merge(df_concordance[["hs6","naics","commodity"]],
                                left_index = True, right_on = "commodity", how = "right", indicator = True)
# trade_conc
trade_conc.shape # Obs: 9,325

(9325, 5)

In [13]:
def concordance_group(df):
    
    #print(df.index)
    
    num_naics = df.shape[0]
    print(num_naics)
    # This tells me how many naics codes are there
    
    if num_naics == 1:
        # if only one (like most), just grab the naics code
        
        max_naics = df.naics.iloc[0]
        
    else:
        # if many, then grab the max trade flows...
        
        max_naics = df.loc[df.total_trade.idxmax()].naics
    
    foo = {"num_naics": [num_naics],
          "naics": [max_naics]}
    
    return pd.DataFrame(foo)

In [14]:
grp = trade_conc.groupby("hs6")
# print(grp.get_group("030211"))
concordance_group(grp.get_group("841391"))

4


,num_naics,naics
0,4,333911


In [17]:
# mc = grp.apply(concordance_group)

1
1
1
1
4
1
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
1
2
2
1
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
6
1
1
6
1
2
2
2
1
6
1
1
1
6
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
6
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
4
2
1
1
1
5
1
1
1
1
1
1
1
1
2
1
1
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
9
6
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
6
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
5
1
10
10
2
1
1
2
1
1
1
1
1
1
2
1
1
1
2
1
1
1
1
1
2
1
1
3
6
5
1
1
1
1
3
3
3
1
1
1
1
1
1
2
1
1
1
1
1
2
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
4
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
2
1
1
1
1
6
1
2
1
1
1
1
1
3
1
1
6
1
1
1
1
1
1
10
1
1
1
1
1
1
1
2
2
1
3
6
3
1
1
2
1
5
2
4
1
1
4
3
2
2
1
1
2
1
2
1
2
2
2
1
1
1
2
1
1
1
1
1
1
2
1
1
1
1
1
3
2
1
1
1
1
5
6
2
2
2
7
1
1
9
2
2
1
2
2
2
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
4
1
1
1
1
1
1
1
3
2
1
1
2
4
1
2
1
2
2
1
2
2
1
1

1
5
1
4
6
2
1
4
2
3
5
2
1
3
1
1
2
1
1
3
1
2
2
2
2
2
1
4
2
1
5
2
2
2
9
2
3
1
1
6
1
1
1
1
1
1
9
3
3
1
1
1
1
4
3
1
1
1
1
1
1
3
1
1
1
4
5
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
6
2
2
2
1
2
1
1
1
3
6
3
3
3
3
3
1
1
1
1
1
3
1
1
5
6
1
2
4
13
7
13
7
1
1
6
1
4
1
2
5
1
2
1
1
1
1
7
2
5
2
1
3
2
4
5
2
3
3
3
2
3
1
5
2
2
1
3
1
1
1
1
1
1
1
3
1
1
2
1
1
1
1
1
1
1
1
2
2
1
1
6
1
2
1
1
3
5
4
8
8
8
3
2
2
2
3
1
1
1
1
1
1
1
1
1
1
6
4
3
1
1
1
2
1
2
1
3
1
1
1
1
1
1
2
3
1
2
1
1
1
1
1
1
3
3
1
1
1
1
1
1
2
1
2
1
1
1
2
1
1
5
1
1
6
1
1
1
1
1
1
1
1
2
1
1
1
8
5
2
1
1
1
1
1
4
1
1
1
3
2
1
6
2
1
1
1
7
1
3
1
2
1
1
1
11
2
1
2
2
2
1
1
1
1
1
1
1
1
1
1
1
2
1
5
1
1
1
1
2
3
2
1
2
1
1
1
1
4
4
5
6
4
1
1
1
2
1
3
1
1
1
1
1
2
1
1
1
1
1
7
1
1
1
3
3
1
1
1
2
9
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
2
2
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
2
1
1
1
1
2
1
5
1
2
2
1
2
1
1
1
1
4
2
1
4
3
1
3
7
2
1
2
2
4
2
1
1
8
4
1
1
1
2
1
1
1
1
2
3
1
1
1
3
1
2
2
3
1
2
1
1
1
1
1
1
4
1
1
1
1
2
2
2
1
1
1
1
1
1
1
1
3
1
1
3
1
1
5
7
3
2
1
1
1
2
1
1
2
1

In [20]:
mc.reset_index(inplace = True)
# mc.head()
mc.shape # Obs: 5,376

(5376, 6)